In [ ]:
# Importación de librerías
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from sklearn.model_selection import (
  KFold,
  GroupKFold,
  StratifiedKFold,
  StratifiedGroupKFold,
  TimeSeriesSplit,
  LeaveOneOut,
  ShuffleSplit,
  StratifiedShuffleSplit
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

# Configuración de estilo para gráficos
plt.style.use('seaborn-v0_8-darkgrid')

# Paletas de colores
cmap_data = plt.cm.tab20    # Mejor paleta para datos categóricos
cmap_cv = plt.cm.RdYlBu     # Mejor contraste para train/test
cmap_groups = plt.cm.Set3   # Paleta para grupos

### Funciones Auxiliares de Visualización

In [ ]:
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm

def visualize_groups(classes, groups, name):
  fig, ax = plt.subplots()
  ax.scatter(
    range(len(groups)),
    [0.5] * len(groups),
    c=groups,
    marker="_",
    lw=50,
    cmap=cmap_data,
  )
  ax.scatter(
    range(len(groups)),
    [3.5] * len(groups),
    c=classes,
    marker="_",
    lw=50,
    cmap=cmap_data,
  )
  ax.set(
    ylim=[-1, 5],
    yticks=[0.5, 3.5],
    yticklabels=["Data\ngroup", "Data\nclass"],
    xlabel="Sample index",
  )

def plot_cv_indices(cv, X, y, group, ax, n_splits, lw=25):
  for ii, (tr, tt) in enumerate(cv.split(X=X, y=y, groups=group)):
    idx = np.array([np.nan] * len(X))
    idx[tt] = 1
    idx[tr] = 0

    ax.scatter(
      range(len(idx)),
      [ii + 0.5] * len(idx),
      c=idx,
      marker="_",
      lw=lw,
      cmap=cmap_cv,
      vmin=-0.2,
      vmax=1.2,
    )

  ax.scatter(range(len(X)), [ii + 1.5] * len(X), c=y, marker="_", lw=lw, cmap=cmap_data)
  ax.scatter(range(len(X)), [ii + 2.5] * len(X), c=group, marker="_", lw=lw, cmap=cmap_data)

  yticklabels = list(range(n_splits)) + ["class", "group"]
  ax.set(
    yticks=np.arange(n_splits + 2) + 0.5,
    yticklabels=yticklabels,
    xlabel="Sample index",
    ylabel="CV iteration",
    ylim=[n_splits + 2.2, -0.2],
    xlim=[0, X.shape[0]],
  )
  ax.set_title("{}".format(type(cv).__name__), fontsize=15)
  return ax

def plot_cv(cv, X, y, groups, n_splits=5, title='Div'):
  # Si cv es una clase, instanciarla apropiadamente
  if isinstance(cv, type):  # Es una clase, no una instancia
    if cv.__name__ == 'LeaveOneOut':
      this_cv = cv()  # LeaveOneOut no acepta n_splits
    else:
      this_cv = cv(n_splits=n_splits)
  else:  # Ya es una instancia
    this_cv = cv
    # Obtener n_splits de la instancia si está disponible
    if hasattr(this_cv, 'n_splits'):
      n_splits = this_cv.n_splits
  
  fig, ax = plt.subplots(figsize=(15, 5))
  plot_cv_indices(this_cv, X, y, groups, ax, n_splits)
  
  ax.legend(
    [Patch(color=cmap_cv(0.8)), Patch(color=cmap_cv(0.02))],
    ["Testing set", "Training set"],
    loc=(1.02, 0.8),
  )
  plt.title(title)
  plt.tight_layout()
  fig.subplots_adjust(right=0.7)
  plt.show()

### Generación de Datos de Ejemplo

In [ ]:
def generate_data():
  n_points = 100
  X_ = np.random.randn(100, 10)

  percentiles_classes = [0.1, 0.9]
  y_ = np.hstack([[ii] * int(100 * perc) for ii, perc in enumerate(percentiles_classes)])

  groups_ = np.hstack([[ii] * 10 for ii in range(10)])
  return X_, y_, groups_

X, y, groups = generate_data()

### Cross-Validation

**Cross-Validation** es una técnica estadística utilizada para evaluar y comparar modelos de ML. En lugar de hacer una única división Train-Test, divide el conjunto de datos múltiples veces para obtener estimaciones más robustas del rendimiento del modelo. 

#### KFold (K-Particiones)
- Divide los datos en $K$ particiones de datos consecutivas con igual tamaño y no es aleatorio el proceso
- En cada iteración: $K-1$ particiones para entrenamiento, $1$ para prueba
- Procesa todas las combinaciones posibles

In [ ]:
plot_cv(
  KFold, X, y, groups, n_splits=5,
  title="KFold"
)

#### StratifiedKFold (K-Particiones Estratificadas)
- Similar a KFold pero mantiene la proporción de clases en cada partición
- Crucial para conjuntos desbalanceados 
- Solo para problemas de clasificación

In [ ]:
plot_cv(
  StratifiedKFold, X, y, groups, n_splits=5,
  title="KFold"
)

#### GroupKFold (K-Particiones por Grupos)
- Garantiza que muestras del mismo grupo estén en una misma partición
- Evita fugas de datos cuando hay dependencias entre muestras
- Útil para datos con múltiples muestras por sujeto/entidad

In [ ]:
plot_cv(
  GroupKFold, X, y, groups, n_splits=5,
  title="GroupKFold"
)

#### StratifiedGroupKFold (K-Particiones Estratificadas por Grupos)
- Combina *stratification* (balance de clases) y *grouping* (grupos intactos)
- Intenta mantener ambos aspectos simultáneamente

In [ ]:
plot_cv(
  StratifiedGroupKFold, X, y, groups, n_splits=5,
  title="StratifiedGroupKFold"
)

#### TimeSeriesSplit (Particiones de Series Temporales) 
- Respeta el orden temporal de los datos
- Cada conjunto de entrenamiento es siempre anterior al de prueba
- Evita usar datos futuros para predecir el pasado

In [ ]:
plot_cv(
  TimeSeriesSplit, X, y, groups, n_splits=5,
  title="TimeSeriesSplit"
)

#### ShuffleSplit (División Aleatoria)
- Random Sampling sobre los datos en cada iteración
- Puede incluir muestras repetidas entre iteraciones
- Controla tamaño de train/test con parámetros

In [ ]:
shuffle_cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
plot_cv(
  shuffle_cv, X, y, groups, n_splits=5,
  title="ShuffleSplit"
)

#### StratifiedShuffleSplit (División Aleatoria Estratificada)
- Versión estratificada de ShuffleSplit
- Mantiene proporción de clases en cada división

In [ ]:
strat_shuffle = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
plot_cv(
  strat_shuffle, X, y, groups, n_splits=5,
  title="StratifiedShuffleSplit"
)

### Esquema Básico de Selección de Técnicas de Validación Cruzada

![](img/cv-techniques%20(1).png)